In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import joblib

# ==========================================
# 1. LOAD THE BRAIN (SERIALIZED MODEL)
# ==========================================
# We don't hardcode numbers anymore. We load them from the file.
model_artifacts = joblib.load('../src/mmm_model.pkl')

model = model_artifacts['model']
features = model_artifacts['features']
sat_params = model_artifacts['saturation_params']

# Extract Coefficients from the Ridge Model
# We map them to channel names so we know which is which
coefs = dict(zip(features, model.coef_))

print("--- LOADED MODEL PARAMETERS ---")
print(f"TV Coefficient:     {coefs['TV_Adstock']:.4f}")
print(f"Social Coefficient: {coefs['Social_Adstock']:.4f}")
print(f"Radio Coefficient:  {coefs['Radio_Adstock']:.4f}")
print("-------------------------------")

# ==========================================
# 2. DEFINE THE "REALITY" FUNCTION (HILL CURVE)
# ==========================================
def hill_saturation(spend, alpha, beta):
    """
    Mathematically models diminishing returns.
    Alpha: Half-saturation point (where curve bends)
    Beta: Slope (how steep the curve is)
    """
    return 1 / (1 + (spend / alpha)**(-beta))

# ==========================================
# 3. DEFINE THE OBJECTIVE FUNCTION
# ==========================================
def calculate_total_sales(budget_allocation):
    """
    Calculates total sales for a given budget split, 
    RESPECTING the saturation curves.
    """
    tv_spend, social_spend, radio_spend = budget_allocation
    
    # 1. Calculate 'Effective' Spend using Saturation (The Pro Move)
    # We use the params we saved in the pickle file
    tv_effective = hill_saturation(tv_spend, sat_params['TV']['alpha'], sat_params['TV']['beta'])
    social_effective = hill_saturation(social_spend, sat_params['Social']['alpha'], sat_params['Social']['beta'])
    radio_effective = hill_saturation(radio_spend, sat_params['Radio']['alpha'], sat_params['Radio']['beta'])
    
    # 2. Multiply by the Model's Coefficients
    # Note: We scale by a factor (e.g. 50k) because our Hill function outputs 0-1, 
    # but our coefficients expect Adstocked dollar values. 
    # In a strict production pipeline, this scaling is automated. 
    # Here, we use the alpha values as a proxy for the "Max Potential Volume".
    
    val_tv = tv_effective * coefs['TV_Adstock'] * sat_params['TV']['alpha']
    val_social = social_effective * coefs['Social_Adstock'] * sat_params['Social']['alpha']
    val_radio = radio_effective * coefs['Radio_Adstock'] * sat_params['Radio']['alpha']
    
    total_sales = val_tv + val_social + val_radio
    
    return -total_sales # Negative because we want to MINIMIZE the negative (aka Maximize positive)

# ==========================================
# 4. RUN THE OPTIMIZER
# ==========================================
total_budget = 50000
budget_bounds = ((0, total_budget), (0, total_budget), (0, total_budget))
budget_constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - total_budget})

# Random starting guess
initial_guess = [10000, 10000, 30000]

result = minimize(
    calculate_total_sales, 
    initial_guess, 
    method='SLSQP', 
    bounds=budget_bounds, 
    constraints=budget_constraints
)

# ==========================================
# 5. VIEW RESULTS
# ==========================================
print("\n--- OPTIMIZED BUDGET ALLOCATION (PRO) ---")
print(f"Total Budget: ${total_budget}")
print(f"TV Spend:     ${result.x[0]:,.2f}")
print(f"Social Spend: ${result.x[1]:,.2f}")
print(f"Radio Spend:  ${result.x[2]:,.2f}")
print("-----------------------------------------")

# Validation Logic
if result.x[2] > 49000:
    print("ALERT: The model still dumps everything into Radio.")
    print("Reason: The Radio Coefficient is likely too dominant relative to saturation.")
else:
    print("SUCCESS: The model recommended a MIXED budget!")
    print("It stopped spending on Radio because it hit the Saturation point.")

--- LOADED MODEL PARAMETERS ---
TV Coefficient:     0.5835
Social Coefficient: 1.3472
Radio Coefficient:  4.9472
-------------------------------

--- OPTIMIZED BUDGET ALLOCATION (PRO) ---
Total Budget: $50000
TV Spend:     $20,656.94
Social Spend: $17,398.94
Radio Spend:  $11,944.12
-----------------------------------------
✅ SUCCESS: The model recommended a MIXED budget!
It stopped spending on Radio because it hit the Saturation point.
